In [ ]:
import os
import sys
import time
import random
import warnings
import collections
from dateutil.relativedelta import relativedelta
from datetime import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

The notebook will find any service type missmatches between memberdoc `analytics_claims` and analytics `cb.claims` and update them in batches of 10000

### Pull Data

In [ ]:
query = """
SELECT
    c.analytics_id
  , st.service_type_description AS ref_table
FROM
    fdw_member_doc.analytics_claims c
    JOIN cb.claims ca ON c.analytics_id = ca.id
    JOIN ref.service_types st ON st.id = ca.service_type_id
WHERE
    c.service_type <> st.service_type_description
limit 10000
"""

In [ ]:
# Connect to your postgres DB
# conn = psycopg2.connect()
pg_conn = cb_utils.get_member_doc_conn()
def update_member_doc():
    # Open a cursor to perform database operations
    cur = pg_conn.cursor()

    # Execute a query
    cur.execute("call junk.update_service_type_update();")

    # Retrieve query results
    # records = cur.fetchall()
    cur.close()
    pg_conn.commit()

In [ ]:
def update_service_types():
    df = cb_utils.sql_query_to_df(query, use_cache=False, verbose=False);
    conn = cb_utils.get_engine(source='member_doc')
    i = 1
    elapsed = 0
    start = time.time()
    while df.shape[0] > 0:
        df.to_sql('claim_service_type_mapping', con=conn, schema='junk', index=False, chunksize=1000, if_exists='append')
        update_member_doc()
        df = cb_utils.sql_query_to_df(query, use_cache=False, verbose=False);
        i += 1
        end = time.time()
        duration = end - start
        elapsed += duration
        print(f'Update #: {i} in {round(duration)}s, {round(elapsed)}s total')
        start = end

In [ ]:
update_service_types();

In [ ]:
update_service_types();

In [ ]:
df = cb_utils.sql_query_to_df(query, use_cache=False);

In [ ]:
conn = cb_utils.get_engine(source='member_doc')

In [ ]:
df.to_sql('claim_service_type_mapping', con=conn, schema='junk', index=False, chunksize=1000, if_exists='append')

In [ ]:
update_member_doc()

In [ ]:
df = cb_utils.sql_query_to_df(query, use_cache=False, verbose=False);

In [ ]:
end  = time.time()
print('%.0fs' % (end - start))

In [ ]:
pg_conn.close()